In [2]:
import sys
import hail as hl
import pandas as pd
import numpy as np

In [2]:
def remove_unnamed(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

NOTE: If Age at MRI is not present at qc.tsv read 	21003-0.0 from ukb45494.csv

## Make SAMPLE QC TABLE

DEFINE INPUT AND OUTPUT

In [3]:
SAMPLE_QC_FILE = "/mnt/i/UKB_DATA/imputed_UKB/qc.tsv"
SAMPLE_QC_TABLE = "/mnt/i/UKB_DATA/imputed_UKB/qc.kt"

READ SAMPLE QC

In [4]:
df = hl.import_table(SAMPLE_QC_FILE)

Initializing Hail with default parameters...


2022-06-03 06:34:58 WARN  Utils:69 - Your hostname, DESKTOP-TVGJHK2 resolves to a loopback address: 127.0.1.1; using 172.24.64.1 instead (on interface eth1)
2022-06-03 06:34:58 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-06-03 06:34:58 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-06-03 06:34:58 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.2
SparkUI available at http://172.24.64.1:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.78-0d33f03a1908
LOGGING: writing to /mnt/d/UKB/GWAS_HAIL/hail-20220603-0634-0.2.78-0d33f03a1908.log
2022-06-03 06:35:01 Hail: INFO: Loading 65 fields. Counts by type:
  str: 65


In [5]:
df.count()

488221

In [6]:
df = df.annotate(PC1 = hl.float64(df['PC1']),
                 PC2 = hl.float64(df['PC2']),
                 PC3 = hl.float64(df['PC3']),
                 PC4 = hl.float64(df['PC4']),
                 PC5 = hl.float64(df['PC5']),
                 PC6 = hl.float64(df['PC6']),
                 PC7 = hl.float64(df['PC7']),
                 PC8 = hl.float64(df['PC8']),
                 PC9 = hl.float64(df['PC9']),
                 PC10 = hl.float64(df['PC10']),
                 age_at_MRI = hl.float64(df['age_at_MRI']),
                 eid = df['eid'],
                 in_white_British_ancestry_subset = df['ethnicity'] == "1.0",
                 used_in_pca_calculation = df['used.in.pca.calculation'] == "1.0",
                 no_excess_relatives = df['excess.relatives']=="0.0",
                 putative_sex_chromosome_aneuploidy = df['putative.sex.chromosome.aneuploidy']=="1.0",
                 isFemale = df['Inferred.Gender'] == '0.0')

In [7]:
df = df.filter(df.in_white_British_ancestry_subset==True)

In [8]:
df.count()

409585

In [9]:
df = df.filter(df.no_excess_relatives==True)

In [10]:
df.count()

276705

In [11]:
df = df.filter(df.used_in_pca_calculation==True)

In [12]:
df.count()

272488

In [13]:
df=df.select('eid',
           'isFemale', # good
           'age_at_MRI', # 1 excluded
           'PC1',
           'PC2',
           'PC3',
           'PC4',
           'PC5',
           'PC6',
           'PC7',
           'PC8',
           'PC9',
           'PC10')

In [16]:
df.count()

272488

In [ ]:
df.show(10)

In [17]:
df.write(SAMPLE_QC_TABLE,overwrite=True)

2022-06-03 06:36:13 Hail: INFO: wrote table with 272488 rows in 5 partitions to /mnt/i/UKB_DATA/imputed_UKB/qc.kt
2022-06-03 06:40:09 Hail: INFO: Loading 65 fields. Counts by type:
  str: 65
2022-06-03 06:40:15 Hail: INFO: wrote table with 276705 rows in 5 partitions to /mnt/i/UKB_DATA/imputed_UKB/qc.kt
2022-06-03 06:43:35 Hail: INFO: Reading table to impute column types
2022-06-03 06:43:37 Hail: INFO: Finished type imputation
  Loading field '' as type int32 (imputed)
  Loading field 'idx' as type int32 (imputed)
  Loading field 'pheno' as type str (imputed)
2022-06-03 06:43:38 Hail: INFO: Coerced sorted dataset
2022-06-03 06:44:59 Hail: INFO: Reading table to impute column types
2022-06-03 06:45:00 Hail: INFO: Finished type imputation
  Loading field '' as type int32 (imputed)
  Loading field 'idx' as type int32 (imputed)
  Loading field 'pheno' as type str (imputed)


## BUILD PIPLINES

In [1]:
import sys
import hail as hl
import pandas as pd

In [2]:
pheno = "edv_ratio"
SAMPLE_QC_TABLE = '/mnt/i/UKB_DATA/imputed_UKB/qc.kt'
kt = hl.read_table(SAMPLE_QC_TABLE)
n_samples = kt.count()

PHESANT_FILE = f"/mnt/i/UKB_DATA/tsv_pheno/{pheno}.tsv"

traits = hl.import_table(PHESANT_FILE,impute=True)
traits = traits.annotate(idx = hl.str(traits.idx), pheno=hl.float64(traits.pheno))


traits=traits.key_by('idx')
kt=kt.key_by('eid')
kt=kt.join(traits, how="inner")

Initializing Hail with default parameters...


2022-06-03 07:08:56 WARN  Utils:69 - Your hostname, DESKTOP-TVGJHK2 resolves to a loopback address: 127.0.1.1; using 172.24.64.1 instead (on interface eth1)
2022-06-03 07:08:56 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-06-03 07:08:57 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-06-03 07:08:57 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.2
SparkUI available at http://172.24.64.1:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.78-0d33f03a1908
LOGGING: writing to /mnt/d/UKB/GWAS_HAIL/hail-20220603-0708-0.2.78-0d33f03a1908.log
2022-06-03 07:09:00 Hail: INFO: Reading table to impute column types
2022-06-03 07:09:02 Hail: INFO: Finished type imputation
  Loading field '' as type str (imputed)
  Loading field 'idx' as type int32 (imputed)
  Loading field 'pheno' as type float64 (imputed)


In [3]:
traits.count()

45307

In [4]:
kt.count()

2022-06-03 07:09:16 Hail: INFO: Coerced sorted dataset
2022-06-03 07:09:17 Hail: INFO: Ordering unsorted dataset with network shuffle


26126

In [5]:
kt.show(10)

2022-06-03 07:09:19 Hail: INFO: Coerced sorted dataset
2022-06-03 07:09:19 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,,,,,,,,,
eid,isFemale,age_at_MRI,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,,pheno
str,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str,float64
"""1000158""",False,4.40e+01,-1.16e+01,2.74e+00,-1.26e+00,1.58e+00,-7.08e+00,-3.52e+00,9.70e-01,-3.36e+00,1.05e+00,1.33e+00,"""idx""",8.81e-01
"""1000243""",False,5.90e+01,-1.39e+01,4.20e+00,-3.87e+00,3.94e-01,1.53e+00,-1.22e+00,-4.83e-01,-2.18e+00,1.72e+00,-7.86e-01,"""1000243""",9.46e-01
"""1000262""",True,5.80e+01,-1.46e+01,5.00e+00,-1.88e+00,4.13e+00,-5.70e-01,5.44e-01,-1.30e+00,1.67e+00,-4.42e+00,1.21e+00,"""1000262""",1.04e+00
"""1000489""",False,6.60e+01,-1.13e+01,4.54e+00,-3.70e+00,2.77e+00,-8.50e+00,-1.19e+00,-1.90e-01,-1.82e+00,8.88e+00,5.65e+00,"""1000489""",9.76e-01
"""1000563""",True,4.90e+01,-1.36e+01,3.44e+00,-1.51e+00,2.02e+00,5.62e+00,-3.65e+00,2.92e+00,-8.56e-01,-2.60e+01,4.15e+00,"""1000563""",1.00e+00
"""1000670""",False,6.80e+01,-1.22e+01,4.78e+00,-1.68e+00,-9.45e-02,-5.13e+00,1.35e+00,-1.17e+00,-2.67e+00,-3.86e+00,8.03e-01,"""1000670""",9.87e-01
"""1001023""",False,5.80e+01,-1.26e+01,4.62e+00,-9.56e-01,9.22e-01,1.01e+00,3.02e+00,-5.58e-01,2.69e-01,-8.24e+00,7.87e-01,"""1001023""",1.04e+00
"""1001525""",False,6.20e+01,-1.28e+01,5.11e+00,-1.49e+00,4.08e+00,9.27e+00,2.14e+00,-8.49e-02,2.08e-01,2.99e+00,-6.30e-01,"""1001525""",1.15e+00


In [6]:
kt.write(f"/mnt/i/UKB_DATA/pipelines/{pheno}.kt",overwrite=True)

2022-06-03 07:10:01 Hail: INFO: Coerced sorted dataset
2022-06-03 07:10:02 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-06-03 07:10:03 Hail: INFO: wrote table with 26126 rows in 5 partitions to /mnt/i/UKB_DATA/pipelines/edv_ratio.kt
2022-06-03 07:11:53 Hail: INFO: Reading table to impute column types
2022-06-03 07:11:55 Hail: INFO: Finished type imputation
  Loading field '' as type str (imputed)
  Loading field 'idx' as type int32 (imputed)
  Loading field 'pheno' as type float64 (imputed)
2022-06-03 07:11:55 Hail: INFO: Coerced sorted dataset
2022-06-03 07:11:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-06-03 07:11:58 Hail: INFO: wrote table with 26126 rows in 5 partitions to /mnt/i/UKB_DATA/pipelines/edv_ratio.kt
2022-06-03 07:11:58 Hail: INFO: Coerced sorted dataset
2022-06-03 07:11:58 Hail: INFO: Ordering unsorted dataset with network shuffle


## ANNOTATE VARIANTS

In [1]:
import hail as hl

In [2]:
GWAS_VARIANTS_VDS = '1_gwas_variants.vds'

In [3]:
vds = hl.read_matrix_table(GWAS_VARIANTS_VDS)

Initializing Hail with default parameters...


2022-06-03 04:57:08 WARN  Utils:69 - Your hostname, DESKTOP-TVGJHK2 resolves to a loopback address: 127.0.1.1; using 172.24.64.1 instead (on interface eth1)
2022-06-03 04:57:08 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-06-03 04:57:10 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-06-03 04:57:12 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.2
SparkUI available at http://172.24.64.1:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.78-0d33f03a1908
LOGGING: writing to /mnt/d/UKB/GWAS_HAIL/hail-20220603-0457-0.2.78-0d33f03a1908.log


In [4]:
vds.show(10)

+---------------+------------+--------------+------------------------------+
| locus         | alleles    | '6017343'.GT | '6017343'.GP                 |
+---------------+------------+--------------+------------------------------+
| locus<GRCh37> | array<str> | call         | array<float64>               |
+---------------+------------+--------------+------------------------------+
| 1:11008       | ["C","G"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 1:11012       | ["C","G"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 1:13110       | ["G","A"]  | 0/0          | [9.96e-01,3.92e-03,0.00e+00] |
| 1:13116       | ["T","G"]  | 0/0          | [9.96e-01,3.92e-03,0.00e+00] |
| 1:13118       | ["A","G"]  | 0/0          | [9.96e-01,3.92e-03,0.00e+00] |
| 1:13273       | ["G","C"]  | 0/0          | [9.88e-01,1.18e-02,0.00e+00] |
| 1:14464       | ["A","T"]  | 0/0          | [9.65e-01,3.53e-02,0.00e+00] |
| 1:14599       | ["T","A"]  | 0/0          | [9.76e-01,2.35e-02,0.00e+00] |
| 1:14604       | ["A","G"]  | 0/0          | [9.76e-01,2.35e-02,0.00e+00] |
| 1:14933       | ["G","A"]  | 0/0          | [9.84e-01,1.57e-02,0.00e+00] |
+---------------+------------+--------------+------------------------------+

+------------------+
| '6017343'.dosage |
+------------------+
|          float64 |
+------------------+
|         0.00e+00 |
|         0.00e+00 |
|         3.92e-03 |
|         3.92e-03 |
|         3.92e-03 |
|         1.18e-02 |
|         3.53e-02 |
|         2.35e-02 |
|         2.35e-02 |
|         1.57e-02 |
+------------------+
showing top 10 rows
showing the first 1 of 22590 columns

## READ BGEN

In [4]:
bgen = hl.import_bgen(path = "/mnt/i/UKB_DATA/imputed_UKB/imputed/ukb22828_c17_b0_v3.bgen",
                      sample_file= "/mnt/i/UKB_DATA/imputed_UKB/imputed/joined.sample", entry_fields=['GT', 'GP','dosage'])

2022-06-13 11:38:56 Hail: INFO: Number of BGEN files parsed: 1
2022-06-13 11:38:56 Hail: INFO: Number of samples in BGEN files: 487409
2022-06-13 11:38:56 Hail: INFO: Number of variants across all BGEN files: 2660711


In [10]:
bgen.show(10)

+---------------+------------+--------------+------------------------------+
| locus         | alleles    | '5048047'.GT | '5048047'.GP                 |
+---------------+------------+--------------+------------------------------+
| locus<GRCh37> | array<str> | call         | array<float64>               |
+---------------+------------+--------------+------------------------------+
| 17:52         | ["C","A"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:56         | ["C","T"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:78         | ["G","C"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:80         | ["G","A"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:92         | ["G","T"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:168        | ["G","A"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:173        | ["C","T"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:174        | ["G","A"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:185        | ["C","G"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
| 17:186        | ["G","A"]  | 0/0          | [1.00e+00,0.00e+00,0.00e+00] |
+---------------+------------+--------------+------------------------------+

+------------------+
| '5048047'.dosage |
+------------------+
|          float64 |
+------------------+
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
|         0.00e+00 |
+------------------+
showing top 10 rows
showing the first 1 of 487409 columns

In [9]:
bgen.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'varid': str
----------------------------------------
Entry fields:
    'GT': call
    'GP': array<float64>
    'dosage': float64
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [14]:
hl.export_bgen(bgen.filter_cols(bgen.s=="5048047"),"PRS_sample.bgen")

2022-06-13 12:30:16 Hail: INFO: while writing:=================>(544 + 1) / 545]
    PRS_sample.bgen.bgen
  merge time: 542.059ms
